![Podmanlogo](Pictures/podman-logo.png)

# Introduction to kubernetes

We already gave a quick explanation of Kubernetes on Podman 101 workshop, here we will go a bit more in depth and explain some core concepts about deploying workloads on Kubernetes. We will not cover architecture, design and infrastructure requirements here; you can check other resources like the kubernetes workshop in HPE Dev platform.

To begin with, lets review the official definition of kubernetes out of [its documentation](https://kubernetes.io/docs/concepts/overview/):

> Kubernetes is a portable, extensible, open source platform for managing containerized workloads and services, that facilitates both declarative configuration and automation. It has a large, rapidly growing ecosystem. Kubernetes services, support, and tools are widely available.

<img src="Pictures/kubernetes-logo.png" width="500">

In practice, kubernetes is a container orchestrator capable of deploying, managing and scale your workloads accross multiple nodes or systems. Compared with Podman, it's design to work in a multi-node environment, being able to deliver high availability of your deployments. There are many other differences in they way both tools work, we'll be looking at some of those during this workshop.

As mentioned above kubernetes is a container orchestrator, meaning it can manage your containerized workloads. But enterprises need more than just an orchestrator, they need a whole platform with monitoring, alerting, role based access control, security features, developer tools, etc. The open source community has created some kubernetes distributions that include those additional capabilities that kubernetes doesn't have out of the box. During this workshop we will be using [Red Hat OpenShift](https://www.redhat.com/en/technologies/cloud-computing/openshift), which is a fully open source solution based on the [OKD](https://www.okd.io/) community project. This platform includes everything you may need to use kubernetes in your organization out of the box.

![OpenShiftLogo](Pictures/openshift-logo.png)

> **Note**: OpenShift uses its own command line tool "oc", but as its based on kubernetes you can also use the "kubectl" command line tool to interact with the cluster for most of the basic operations. The "oc" tool adds the capability of interacting with OpenShift features that are not included with kubernetes out of the box, for this workshop we'll run basic operations so both could be used indistinguibly. As we're used to work with "oc" we chose to use this tool for the rest of the workshop.

# Kubernetes core concepts

Kubernetes uses different entities or objects to manage containerized workloads, in this section we'll review the basic ones. These core concepts are the following:

 - **Pod**: Pods are the smallest deployable units of computing that you can create and manage in Kubernetes. A Pod is a group of one or more containers, with shared storage and network resources, and a specification for how to run the containers. A Pod's contents are always co-located and co-scheduled, and run in a shared context.
 - **ReplicaSet**: A ReplicaSet's purpose is to maintain a stable set of replica Pods running at any given time. As such, it is often used to guarantee the availability of a specified number of identical Pods. By default kubernetes will try to spread replicas across different nodes of the cluster to gain high availability.
 - **Deployment**: A Deployment provides declarative updates for Pods and ReplicaSets. In other words, is an object that can manage Pods and ReplicaSets and the way they will be updated when a new version of the application is available.
 - **Service**: a Service is a method for exposing a network application that is running as one or more Pods in your cluster. Fot example, if you have an application with 5 replicas you will not need to remember any ip or dns name for each of those replicas. Instead the Service will act as a network endpoint that, once reached, redirects and load balances the traffic to those endpoints. You could think of it as a forward proxy.
 - **PersistentVolume (PV)**: is a piece of storage in the cluster that has been provisioned by an administrator or dynamically provisioned using Storage Classes.
 - **Namespaces**: a mechanism for isolating groups of resources within a single cluster. Names of resources need to be unique within a namespace, but not across namespaces. Namespace-based scoping is applicable only for namespaced objects (e.g. Deployments, Services, etc) and not for cluster-wide objects (e.g. StorageClass, Nodes, PersistentVolumes, etc).

> **Note**: part of these definitions were taken directly from [the kubernetes documentation](https://kubernetes.io/docs/concepts/workloads/pods/)

Lets see this in practice. First log into the system connected to the cluster:

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Podman201'] }}

First thing we want to do is login into the cluster.

In [ ]:
oc login -u student{{ STDID}} -p {{ PASSSTU }} --insecure-skip-tls-verify https://{{ OCENDPOINT }}:6443

Once we have access to the cluster we can run basic commands, like whoami to show our user:

In [ ]:
oc whoami

Or get the link to the web console (it's not accessible through the internet for this workshop):

In [ ]:
oc whoami --show-console

> **Important note**: The result of this command will provide a link to the OpenShift web console. However , due to firewalling restrictions, this link will not be accessible from here.

Before we deploy anything we need to have a namespace, that is a logical space in which our resources are going to be deployed. OpenShift can manage namespaces, but it also has the "project" object. A project is essentially the same as a namespace, but OpenShift provides additional administrative controls for them, we'll not go into detail in this workshop, for now just think of a project as a namespace.

Lets create our first project:

In [ ]:
oc new-project my-project-student{{ STDID }}

The project was created and now you're working on it, meaning every resource you create will be deployed in this project. We'll later show how to work with multiple projects.

We can start deploying our containers now. Lets begin by running the following:

In [ ]:
oc create deployment my-deployment --replicas 1 --image docker.io/openshift/hello-openshift

When we create a deployment object it automatically creates several objects: one or more Pods (one per replica), a ReplicaSet and a Deployment. Lets review all of these resources.

Run the following command to gain an overview:

In [ ]:
oc get all

Run the following command to see all the pods available in the current project:

In [ ]:
oc get pods

You see the pod has the same name as the deployment followed by a random set of numbers and letters, this will be important in next steps.

You can use the describe command to see the details of the pod:

In [ ]:
export PODNAME=$(oc get pod -l app=my-deployment -o jsonpath="{.items[0].metadata.name}")
oc describe pod $PODNAME

As you can see it outputs all the details about our newly created Pod. It's important to note that, as everytime you create a Pod it receives a random name, we have used a reference to the output of a different command using envrionment variables.

Look at the command we used to get the Pod name: oc get pod -l app=my-deployment -o jsonpath="{.items[0].metadata.name}". It's just a "oc get" command like the one we run before. But this time we used the "-l" parameter because we only want to get the pods with the label "app" and a value of "my-deployment" for it. Be aware that OpenShift automatically created that label so we can easily find and work with our workload. We also used the "-o" option to format the output, every object in kubernetes and its status is represented by a file that you can extract in yaml or json format, we exported it in json format and only wanted the name field so we used the structure within brackets to filter it.

You can get the yaml or json file for any resource on the cluster, lets extract the yaml for our Pod:

In [ ]:
export PODNAME=$(oc get pod -l app=my-deployment -o jsonpath="{.items[0].metadata.name}")
oc get pod $PODNAME -o yaml

Kubernetes will export the yaml file of any object when you add the "-o yaml" option to any "oc get" command. We'll go into details of yaml files later.

As mentioned before you can also see the ReplicaSet object that you created when the Deployment was created:

In [ ]:
oc get replicaset

The ReplicaSet is in charge of maintaining the proper amount of replicas for our application, that's why the output shows the amount of desired replicas, current replicas and ready replicas. You can modify the replicas with the "oc scale" command pointing to the deployment:

In [ ]:
oc scale deployment/my-deployment --replicas 3

Take a look at the command and the way we referenced the deployment by using the "object-type/object-name" naming convention.

Get the ReplicaSet again:

In [ ]:
oc get replicaset

As you can see we have now three replicas of our container, by default kubernetes will try to spread all your replicas across different nodes to provide high availability.

You can also get the deployment information with:

In [ ]:
oc get deployments

You can also see the amount of replicas through the Deployment object.

The "oc describe" works for all the object types in kubernetes, lets get the information about our deployment:

In [ ]:
oc describe deployment my-deployment

For the moment we have seen the Deployment works as a wrapper for Pods and ReplicaSet object, in a single one. But it's much more, it also helps managing your update strategy. Look at the output of previous command, specifically where it says "StrategyType: RollingUpdate" and "RollingUpdateStrategy:  25% max unavailable, 25% max surge". That is the update strategy that kubernetes uses by default for Deployments but can be modified.

There are mainly two update strategies you can use:
 - Rolling update: consist in updating a percentage of your replicas to the new version of the application, once those have finished the update the Deployment will update other replicas and so on. You can choose a percentage of maximum unavailability of replicas, by default is 25% meaning that a maximum of 25% of your replicas will update at a time (because there will be a super small amount of unavailability time when you update your container to a new version).
 - Recreate: the deployment will kill all your replicas, once all are down it will recreate all of them at once with the new version.

 > **Note**: if you want to learn more about update strategies take a look at [official kubernetes documentation](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/)

 Sometimes you may want to force a redeploy of your application (usually called rollout), you can easily do it:

In [ ]:
oc rollout restart deployment my-deployment

We've seen you can create different amount of replicas, each replica is a pod and it receives a name based on the deployment name plus some random letters and numbers. Now lets see what happens when we manually delete a Pod.

First look at the name of your pods:

In [ ]:
oc get pods

Now delete one of them:

In [ ]:
export PODNAME=$(oc get pod -l app=my-deployment -o jsonpath="{.items[0].metadata.name}")
oc delete pod $PODNAME

And get the pods again:

In [ ]:
oc get pods

Observe you have again three pods, but now one of them has changed its name. This is because it's a different pod, you have killed the previous pod and the ReplicaSet has created a new one from the same image because that's how it's configured, it's going to do as much as it can to have three pods. The important point here is that pods are ephemeral, meaning you can delete them and nothing should change because the ReplicaSet will recreate them. The same happens when you update the container image to a new version and it's rolled out, the platform will actually create new pods with the new version of the application.

Pods changing name all the time can be a problem for networking. If the name could change at any moment, how can one microservice or application reach the others? This is achieved with the Service object, it creates a network endpoint internal to the cluster so other application or services can reach it. The Service will receive the traffic and load balance it to the different replicas of the attached Deployment. All of this is automatically managed by the Software Defined Networking in kubernetes. Let see how this works.

Create a service attached to our Deployment with the "oc expose" command:

In [ ]:
oc expose deployment/my-deployment --port 8080

The service has been create and it's exposing our Pods in the port 8080. Any other application can reach this microservice pointing to its DNS name which is the same as the service name.

> **Note**: there are tools in kubernetes to microsegmentate the network, like NetworkPolicy object. If an exposed microservice is not reachable may be because of some additional configuration of the cluster and not because of the Service.

You can run the same generic commands we run before for the Services object:

In [ ]:
oc get services

Now the microservice is exposed, but only inside the cluster. You can also expose a Service which means it'll be reachable from outside of the cluster. Lets do it:

In [ ]:
oc expose service/my-deployment

When you expose a Service it actually creates an object called Route which is the one managing how the application can be reached from outside of the cluster. You can get the http address:

In [ ]:
oc get route

Test the connection works running a curl command:

In [ ]:
export ROUTEADDRESS=$(oc get route my-deployment -o jsonpath={.spec.host})
curl -s $ROUTEADDRESS

You see the "Hello OpenShift!" message from the application, demonstrating everything works as expected.

# Declarative and imperative ways of working

We've seen how to deploy our application using the "oc" command line tool and we also have seen that kubernetes generates yaml files for all the resources we have created. This is because kubernetes can work in two ways: imperative and declarative. But what is the difference?

- Imperative means that you describe the configuration of a resource by executing a command from a terminal.
- Declarative means you create a file describing the configuration for the particular resource and then apply the content of the file to the Kubernetes cluster. 

Until now we have used the imperative way of working, but we could create a yaml file describing how we want our resources to be deployed, apply it to the cluster and it will make sure it's deployed.

We're going to create a new project for our declarative deployment.

In [ ]:
oc new-project declarative-student{{ STDID }}

Now create a yaml file describing our Deployment

In [ ]:
cat << EOF > my-deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-deployment
  namespace: declarative-student{{ STDID }}
spec:
  replicas: 3
  selector:
    matchLabels:
      app: my-deployment
  strategy:
    rollingUpdate:
      maxSurge: 25%
      maxUnavailable: 25%
    type: RollingUpdate
  template:
    metadata:
      labels:
        app: my-deployment
    spec:
      containers:
      - image: docker.io/openshift/hello-openshift
        imagePullPolicy: Always
        name: hello-openshift
EOF
cat my-deployment.yaml

As you can see the yaml file describes all the details about the Deployment that we've been discussing during the rest of the workshop, the only change is the namespace as we are working in a different project. We can apply it running:

In [ ]:
oc apply -f my-deployment.yaml

You can see everything has been deployed:

In [ ]:
oc get deployment

As you can see it works with a very simple command. The kubernetes cluster stores your file and does as much as possible to guarantee your application is deployed as described in this file. You can modify the content of the file stored by kubernetes with imperative commands, as we've seen before, and then export your modified yaml file running the command "oc get object-type object-name -o yaml".

All the objects work this same way in kubernetes, create the Service and Route yaml file.

In [ ]:
cat << EOF > my-deployment-service-route.yaml
apiVersion: v1
kind: Service
metadata:
  labels:
    app: my-deployment
  name: my-deployment
  namespace: declarative-student{{ STDID }}
spec:
  ports:
  - port: 8080
    protocol: TCP
    targetPort: 8080
  selector:
    app: my-deployment
  type: ClusterIP
---
apiVersion: route.openshift.io/v1
kind: Route
metadata:
  labels:
    app: my-deployment
  name: my-deployment
  namespace: declarative-student{{ STDID }}
spec:
  port:
    targetPort: 8080
  to:
    kind: Service
    name: my-deployment
EOF
cat my-deployment-service-route.yaml

It's possible to have a single file with more than one object defined within it, just separate the definition of each object with "---".

Again, you can see the object for the route and the service are just the way we discussed when using the imperative commands but in a file. There are some additional details but we will not cover those during this workshop.

Apply the new file:

In [ ]:
oc apply -f my-deployment-service-route.yaml

If you remember we have now two projects, you can switch projects with the command "oc project":

In [ ]:
oc project my-project-student{{ STDID }}

Now you are on the "my-project-student123" project. To switch back to the project "declarative-student123" just use the same command but pointing to the project name:

In [ ]:
oc project declarative-student{{ STDID }}

This isn't very handy, so remember you can use the option "-n" with any "oc" command to define to which project the command needs to be applied. Next command will show the pods in project "declarative-student{{ STDID }}"

In [ ]:
oc get pods -n declarative-student{{ STDID }}

While the next one will show it for the project "my-project-student{{ STDID }}":

In [ ]:
oc get pods -n my-project-student{{ STDID }}

Use these commands to check if the app is reachable via http from both projects. First project:

In [ ]:
export ROUTEADDRESS=$(oc get route my-deployment -n my-project-student{{ STDID }} -o jsonpath={.spec.host})
curl -s $ROUTEADDRESS

Second project:

In [ ]:
export ROUTEADDRESS=$(oc get route my-deployment -n declarative-student{{ STDID }} -o jsonpath={.spec.host})
curl -s $ROUTEADDRESS

With this we've seen the basics of how to deploy an application to kubernetes, we will see how to deploy the Patient Portal application in next section of the workshop.

# Cleanup

In [ ]:
oc delete project my-project-student{{ STDID }}
oc delete project declarative-student{{ STDID }}
rm my-deployment-service-route.yaml my-deployment.yaml

In [ ]:
%logout

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#631f61;"></i>&nbsp;&nbsp;Next Steps

# Lab  : From Podman to Kubernetes

<h2>Next LAB&nbsp;&nbsp;&nbsp;&nbsp;<a href="6-WKSHP-From-Podman-to-kubernetes.ipynb" target="New" title="Next LAB: From Podman to Kubernetes"><i class="fas fa-chevron-circle-right" style="color:#631f61;"></i></a></h2>

</br>
 <a href="4-WKSHP-Running-containers-at-system-start.ipynb" target="New" title="Back: Running containers at system start"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#631f61;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="6-WKSHP-From-Podman-to-kubernetes.ipynb" target="New" title="Next:From Podman to Kubernetes"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#631f61;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
